# Binary Sentiment Analysis of French Movie Reviews

### Objectives
1. Text cleaning
2. Text preprocessing for custom embedding Neural Network
3. Train RNN model for sentiment analysis

⚠️ This notebook will be your final deliverable. 
- Make sure it can run "restart and run all"
- Delete useless code cells
- Do not "clear output"

# 0. Load data

Our dataset contains 30,000 french reviews of movies, along with the binary class 1 (positive) or 0 (negative) score

In [1]:
import pandas as pd
import numpy as np

In [2]:
# We load the dataset for you
data = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies.csv')
data

review  polarity
0      Ça commence à devenir énervant d'avoir l'impre...         0
1      J'ai aimé ce film, si il ressemble a un docume...         1
2      Une grosse merde ce haneke ce faire produire p...         0
3      Beau mélodrame magnifiquement photographié, "V...         1
4      A la poursuite du diamant vers est un film pro...         1
...                                                  ...       ...
29946  Le meilleur film de super-héros derrière le ba...         1
29947  Un drame qui est d'une efficacité remarquable....         1
29948  Une daube hollywoodienne de plus, aucun intérê...         0
29949  Et voilà un nouveau biopic sur la star du X Li...         0
29950  Un film qui fait vieux, avec des acteurs pas t...         0

[29951 rows x 2 columns]

In [3]:
# We create features
y = data.polarity
X = data.review

# We analyse class balance
print(pd.value_counts(y))

1    15051
0    14900
Name: polarity, dtype: int64


In [4]:
# We check various reviews
print(f'polarity: {y[0]} \n')
print(X[0])

polarity: 0 

Ça commence à devenir énervant d'avoir l'impression de voir et revoir le même genre de film à savoir : la comédie romantique, surement le genre le plus prolifique de le production française actuelle. Le problème c'est que l'on a souvent affaire à des niaiseries de faible niveau comme celui ci. Avec un scenario ultra balisé et conventionnel, c'est à se demander comment ça peut passer les portes d'un producteur. Bref cette sempiternel histoire d'un homme mentant au nom de l'amour pour reconquérir une femme et qui à la fin se prend son mensonge en pleine figure est d'une originalité affligeante, et ce n'est pas la présence au casting de l'ex miss météo Charlotte Le Bon qui rêve surement d'avoir la même carrière que Louise Bourgoin qui change la donne.


# 1. Clean Text

❓ We need to give a _quick & dirty_ cleaning to all the sentences in the dataset. Create a variable `X_clean` of similar shape, but with the following cleaning:
- Replace french accents by their non-accentuated equivalent using the [unidecode.unidecode()](https://pypi.org/project/Unidecode/) method
- Reduce all uppercases to lowercases
- Remove any characters outside of a-z, for instance using `string.isalpha()`

😌 You will be given the solution `X_clean` in the next question to make sure you can complete the challenge

In [5]:
from unidecode import unidecode

In [6]:
# function that returns a string will all the prerequisites of the exercise
def clean(text):
    text = unidecode(text)
    text = text.lower()
    text = ''.join(word for word in text if not word.isdigit())
    return text

In [7]:
X_clean = X

In [8]:
for i in range(len(X_clean)):
    X_clean[i] = clean(X_clean[i])
    

<ipython-input-8-771f6a716f26>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_clean[i] = clean(X_clean[i])


In [9]:
X_clean[29950]

"un film qui fait vieux, avec des acteurs pas tout jeunes, pour une histoire des annees /. la french est un produit hors d'age, qui joue sur tous les tons la po(au)se vintage. c'est franchement pas affriolant, comme un dimanche apres-midi chez drucker qui enqueterait sur le milieu marseillais. dujardin n'est pas tres bon (mais peut-il l'etre ?), lellouche est un peu plus credible. a porter au credit du film : de tres bons seconds roles, un caractere documentaire instructif pour les jeunes generations, une jolie photographie de marseille. a decharge : le reste. engonce dans sa mollesse intrinseque, le film parvient a susciter plus l'ennui curieux que le degout, ce qui n'est deja pas si mal. aux  articles :"

In [32]:
from nbresult import ChallengeResult

result = ChallengeResult('C14',
    shape = X_clean.shape,
    first_sentence = X_clean[0]
)
result.write()

# 2. Preprocess data

Now that we have clean sentences, we need to convert each one into a list of integers of fixed size
- For example, the sentence: `"this was good"` should become something like `array([1, 3, 18, 0, 0, 0, ...0], dtype=int32)` where each integer match to a each _unique_ word in your corpus of sentences.

❓ Create a numpy ndarray `X_input` of shape (29951, 100) that will be the direct input to your Neutral Network. 

- 29951 represents the number of reviews in the dataset `X_clean`
- 100 represents the maximum number of words to keep for each movie review.
- It must contain only numerical values, without any `NaN`
- In the process, compute and save the number of _unique_ words in your cleaned corpus under `vocab_size` variable

👉 First, you **must** start back from the clean solution below (14Mo)

In [88]:
X_clean = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_clean.csv")['review']
X_clean

0        ca commence a devenir enervant de voir et revo...
1        aime ce film si il ressemble a un documentaire...
2        une grosse merde ce haneke ce faire produire p...
3        beau melodrame magnifiquement photographie ver...
4        a la poursuite du diamant vers est un film pro...
                               ...                        
29946    le meilleur film de derriere le batman de nola...
29947    un drame qui est efficacite remarquable un fil...
29948    une daube hollywoodienne de plus aucun interet...
29949    et voila un nouveau biopic sur la star du x li...
29950    un film qui fait vieux avec des acteurs pas to...
Name: review, Length: 29951, dtype: object

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
# initializing tokenizer

In [89]:
maxlen = 100
maxwords = 10000

In [90]:
def tokeninit(train, maxwords):
  tokenizer = Tokenizer(num_words=maxwords)
  tokenizer.fit_on_texts(train)
  return tokenizer

In [91]:
tokenizer = tokeninit(X_clean, maxwords)

In [92]:
len(tokenizer.word_index)

62379

In [76]:
vocab_size = 62379

In [93]:
X_clean = np.array(X_clean)

In [19]:
# defining the function to turn the texts into lists of 100 integers

In [94]:
def getSequenceFromArray(array, tokenizer, maxlen):
  sequences = tokenizer.texts_to_sequences(array)
  return pad_sequences(sequences, maxlen=maxlen)
    

In [95]:
X_input = getSequenceFromArray(X_clean, tokenizer, maxlen)

In [96]:
X_input.shape

(29951, 100)

In [63]:
from nbresult import ChallengeResult

result = ChallengeResult('C1415',
    type_X = type(X_input),
    shape = X_input.shape, 
    input_1 = X_input[1], 
)
result.write()

# 3. Neural Network

❓Create and fit a Neural Netork that takes `X_input` and `y` as input, to binary classify each sentence's sentiment

- You cannot use transfer learning or other pre-existing Word2Vec models
- You must use a "recurrent" architecture to _capture_ a notion of order in the sentences' words
- The performance metrics for this task is "accuracy"
- Store your model in a variable `model` 
- Store the result your `model.fit()` in a variable `history`. 
- ⚠️ `history.history` must comprises a measure of the `val_accuracy` at each epoch.
- You don't need to cross-validate your model

😌 Don't worry, you will not be judged on your computer power: You should be able to reach accuracy significantly better than baseline in less than 3 minutes even without GPUs.

👉 But first, you **must** start back from the solution below (70Mo)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.layers import GRU
from tensorflow.keras import regularizers

In [98]:
url = 'https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_input.csv'
X_input = np.genfromtxt(url, delimiter=',', dtype='int32')

In [99]:
X_input

array([[   15,    16,    17, ...,    84,    15,    85],
       [   86,    72,    13, ...,     0,     0,     0],
       [   59,   102,   103, ...,     0,     0,     0],
       ...,
       [   59,  2439,  5003, ...,     0,     0,     0],
       [ 6207, 41774,  3201, ...,     3,   198,  9426],
       [  237,   806,    57, ...,   793,   111,  3154]], dtype=int32)

In [112]:
# initializing

model = Sequential()
model.add(Embedding(input_dim=62380, output_dim=100))
model.add(GRU(150, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, None, 100)         6238000   
_________________________________________________________________
gru_18 (GRU)                 (None, 150)               113400    
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 151       
Total params: 6,351,551
Trainable params: 6,351,551
Non-trainable params: 0
_________________________________________________________________


In [113]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [108]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

In [109]:
X_input.shape

(29951, 100)

In [110]:
y.shape

(29951,)

In [114]:
history = model.fit(X_input, y,
          validation_split=0.3, 
          batch_size=32,
          epochs=100,
        callbacks=[es])

Epoch 1/100
656/656 [==============================] - 185s 278ms/step - loss: 0.6738 - accuracy: 0.5434 - val_loss: 0.5292 - val_accuracy: 0.7306
Epoch 2/100
656/656 [==============================] - 166s 253ms/step - loss: 0.3170 - accuracy: 0.8836 - val_loss: 0.2562 - val_accuracy: 0.8974
Epoch 3/100
656/656 [==============================] - 156s 237ms/step - loss: 0.1780 - accuracy: 0.9334 - val_loss: 0.2251 - val_accuracy: 0.9116
Epoch 4/100
656/656 [==============================] - 166s 253ms/step - loss: 0.1358 - accuracy: 0.9507 - val_loss: 0.2244 - val_accuracy: 0.9112
Epoch 5/100
656/656 [==============================] - 170s 260ms/step - loss: 0.1070 - accuracy: 0.9616 - val_loss: 0.2285 - val_accuracy: 0.9089
Epoch 6/100
656/656 [==============================] - 182s 277ms/step - loss: 0.0943 - accuracy: 0.9663 - val_loss: 0.2617 - val_accuracy: 0.9084
Epoch 7/100
656/656 [==============================] - 185s 282ms/step - loss: 0.0705 - accuracy: 0.9774 - val_loss: 0

In [116]:
from nbresult import ChallengeResult
result = ChallengeResult('C1517',
                         history=history.history)
result.write()